In [96]:
using Convex, SCS, Images, DataFrames, Colors, FixedPointNumbers, ImageView
set_default_solver(SCSSolver(verbose=0)) 

SCS.SCSSolver(Any[(:verbose,0)])

In [83]:
k = 45
trial = 2

# load Data
path = "./HeadsAndTails/"
files = readdir(path)
n = 100
finalDim = (20,20)
fileInd = rand(1:size(files,1),n)
curfile = joinpath(path,files[1])
temp = convert(Array{Float64,2},convert(Image{Gray},load(curfile)))
temp = Images.imresize(temp,finalDim)
dim = size(temp)
D = zeros(prod(dim), n)
D[:,1] = vec(temp)
for i=2:n
    curfile = joinpath(path,files[fileInd[i]])
    temp = convert(Array{Float64,2},convert(Image{Gray},load(curfile)))
    temp = Images.imresize(temp,finalDim)
    D[:,i] = vec(temp)
end

# Normalize
x_bar = zeros(size(D,1),1)
for i =1:size(D,2)
    x_bar = x_bar + D[:,i]
end
x_bar = x_bar/size(D,2)

# Get residuals
X_tilde = D - repmat(x_bar ,1,size(D,2))

covm = X_tilde'*X_tilde
U, S, V = svd(covm)
Vk = zeros(size(V,1),k)
Vk = V[:,1:k]
Uk = X_tilde*Vk
norms = sqrt(diag(Uk'*Uk))

A = Uk./ repmat(norms',size(Uk,1),1)

# Project
Px = zeros(k,size(D,2))
for i=1:n
    Px[:,i] = A'*X_tilde[:,i]
end
;

In [84]:
function projtest(A,X,Px, threshold)
    p = A'*vec(X)
    n = size(Px,2)
    fish = false
    snorm = 0
    for i=1:n
        snorm+= norm(p-Px[:,i]) 
    end
    anorm = snorm/n
    if anorm <threshold
        return true
    else
        return false
    end
end

projtest (generic function with 2 methods)

In [85]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test7.jpg")), (240,240)))
save("test1.png",testim)

In [86]:
L = 240
w = 20
testOut = zeros(L,L)
for i in 1:w:L-w+1
    for j in 1:w:L-w+1
        sample = testim[i:i+w-1,j:j+w-1]
        fish = projtest(A,sample,Px,10)
        if fish
            testOut[i:i+w-1,j:j+w-1] = 1
        end
    end
end
    
grayim(testOut)
r = round(Int,prod(size(testOut)))
for i = 1:r
    if testOut[i] == 1
        testOut[i]=testim[i]
    end
end
grayim(testOut)
save("pca1.png",testOut)        

In [87]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test6.jpg")), (240,240)))
save("test2.png",testim)

In [88]:
L = 240
w = 20
testOut = zeros(L,L)
for i in 1:w:L-w+1
    for j in 1:w:L-w+1
        sample = testim[i:i+w-1,j:j+w-1]
        fish = projtest(A,sample,Px,10)
        if fish
            testOut[i:i+w-1,j:j+w-1] = 1
        end
    end
end
    
grayim(testOut)
r = round(Int,prod(size(testOut)))
for i = 1:r
    if testOut[i] == 1
        testOut[i]=testim[i]
    end
end
grayim(testOut)
save("pca2.png",testOut)        

In [93]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test3.jpg")), (240,240)))
save("test3.png",testim)

In [90]:
L = 240
w = 20
testOut = zeros(L,L)
for i in 1:w:L-w+1
    for j in 1:w:L-w+1
        sample = testim[i:i+w-1,j:j+w-1]
        fish = projtest(A,sample,Px,10)
        if fish
            testOut[i:i+w-1,j:j+w-1] = 1
        end
    end
end
    
grayim(testOut)
r = round(Int,prod(size(testOut)))
for i = 1:r
    if testOut[i] == 1
        testOut[i]=testim[i]
    end
end
grayim(testOut)
save("pca3.png",testOut)

In [94]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test1.jpg")), (240,240)))
save("test4.png",testim)

In [92]:
L = 240
w = 20
testOut = zeros(L,L)
for i in 1:w:L-w+1
    for j in 1:w:L-w+1
        sample = testim[i:i+w-1,j:j+w-1]
        fish = projtest(A,sample,Px,10)
        if fish
            testOut[i:i+w-1,j:j+w-1] = 1
        end
    end
end
    
grayim(testOut)
r = round(Int,prod(size(testOut)))
for i = 1:r
    if testOut[i] == 1
        testOut[i]=testim[i]
    end
end
grayim(testOut)
save("pca4.png",testOut)        

# Sparse Matrix 

In [105]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test7.jpg")), (220,220)))
Y = testim
X = Variable(size(testim))
E = Variable(size(testim))
lam = 1/sqrt(maximum(size(Y)))

problem = minimize(lam*norm(vec(X),1)+nuclearnorm(E),X+E==Y)
solve!(problem)

In [107]:
grayim(X.value)
save("sprs1.png",map(Images.Clamp01NaN(X.value), X.value))

In [108]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test6.jpg")), (220,220)))
Y = testim
X = Variable(size(testim))
E = Variable(size(testim))
lam = 1/sqrt(maximum(size(Y)))

problem = minimize(lam*norm(vec(X),1)+nuclearnorm(E),X+E==Y)
solve!(problem)

In [109]:
grayim(X.value)
save("sprs2.png",map(Images.Clamp01NaN(X.value), X.value))

In [110]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test3.jpg")), (220,220)))
Y = testim
X = Variable(size(testim))
E = Variable(size(testim))
lam = 1/sqrt(maximum(size(Y)))

problem = minimize(lam*norm(vec(X),1)+nuclearnorm(E),X+E==Y)
solve!(problem)

In [111]:
grayim(X.value)
save("sprs3.png",map(Images.Clamp01NaN(X.value), X.value))

In [112]:
testim = convert(Array{Float64,2},Images.imresize(convert(Image{Gray},load("./Test/test1.jpg")), (220,220)))
Y = testim
X = Variable(size(testim))
E = Variable(size(testim))
lam = 1/sqrt(maximum(size(Y)))

problem = minimize(lam*norm(vec(X),1)+nuclearnorm(E),X+E==Y)
solve!(problem)

In [113]:
grayim(X.value)
save("sprs4.png",map(Images.Clamp01NaN(X.value), X.value))